In [1]:
%pip install python-constraint

Note: you may need to restart the kernel to use updated packages.


In [4]:
from constraint import Problem  # the constraint package
from itertools import permutations # for generating match pairs

## Input Format

In [6]:
# the list of teams
teams = [ "A", "B", "C", "D" ]
# matrix that lists available locations to organize the matches on
location_available = {
    "A": [ 1,       4,    6, 7, 8,    10 ], # location of team A is available for the listed time slots
    "B": [    2,    4, 5, 6, 7,    9,    ],
    "C": [    2, 3,    5,       8,    10 ],
    "D": [ 1,    3,       6,    8, 9, 10 ],
}
# NOTE: this is just an example problem, your implementation should work for any scheduling problem with this structure

In [ ]:
problem = Problem()

# TODO add CSP variables to the problem

def create_match(teams: list[str]) -> list[tuple[str, str]]:
    return list(permutations(teams, 2))

# the function gets the list of team names (strings) as an input 
# the output is a list of tuples, where each tuple represents a match as (home-team, away-team) --> [("A", "B"), ("A", "C"), ...]
# the function returns a list all possible pairs of the teams of the length 2
# we use permutations because it gives us all orderes pairs without repetition
# the combinations function from itertools would give us unordered pairs and just one of the two possible orders for each pair

matches = create_match(teams) # we create the matches using the function defined above

for match in matches:
    problem.addVariable(match, location_available[match[0]])

# match is our game-tupel (home-team, away-team)
# we iterate throuch all matches 
# in each iteration, we add the available timeslots for the home-team as the domain of the variable representing that match

# in this case the constraint 4 "Every pairing {ti,tj} should be played once in ti and once in tj" is already satisfied
# because we created matches using permutations, which gives us both (A, B) and (B, A) for each pair of teams A and B
# when we iterate through matches, we add both matches to the problem with their respective domains

## Constraints

Model the constraints speficied in the exercise here. Add a section for each constraint.

C1: Every team should play every other team exactly twice, once in the first half of the season and another time in the second half of the season.

In [ ]:
# TODO model a constraint here

all_slots = set()   # we create a set to store all available time slots and to avoid duplicates
for slots in location_available.values():   # we iterate through the values of the location_available dictionary
    all_slots.update(slots) # we update the set with the available slots from each team

# a set is unordered, if we divide it direclty, the two halves may not be in order
all_slots = sorted(all_slots)  # by sorting the set, the first half will contain the earlier time slots and the second half the later ones

mid = len(all_slots) // 2  # we calculate the midpoint of the list of time slots
first_half = set(all_slots[:mid])  # we slice the set to get the first half of the season, set is O(1) for lookups, list would be O(n)
second_half = set(all_slots[mid:])  # we slice the set to get the second half

def matches_in_different_halves(slot1, slot2):   # in this function we ensure that A vs B is played in a different half of the season than B vs A
    return ((slot1 in first_half and slot2 in second_half) or
            (slot1 in second_half and slot2 in first_half))

# we iterate through all unique pairs of teams 
for i in range(len(teams)): 
    for j in range(i + 1, len(teams)): # + 1 to avoid self-pairing and duplicate pairs
        # here we create the two matches for each pair of teams
        match1 = (teams[i], teams[j]) # e.g., (A, B)
        match2 = (teams[j], teams[i]) # e.g., (B, A)
        # add constraint: the two matches must be in different halves of the season
        problem.addConstraint(matches_in_different_halves, [match1, match2])

# constraint 5: "The first half of the season must be completed for all teams before the first match of the second half of the season is played."
# constraint 5 is fulfilled by the way we defined the halves of the season above
# first_half and second_half are defined by the sorted order of all available time slots
# since we only allow matches to be scheduled in their respective halves, 
# all matches in the first half must occur before any match in the second half

C2: In every time slot, every team can have at most one match.

In [12]:
from constraint import AllDifferentConstraint

# we have to ensure for eeach team that they play just once in a time slot
for team in teams:
    team_matches = [] # here we store all matches involving the current team
    for match in matches: # we go through all matches
        if team in match: # if our current team is in the match
            team_matches.append(match) # we add the match to the list of all matches for that team
    
    # AllDifferentConstraint ensures that all variables in the list take different values
    # so no two matches involving the same team can be scheduled at the same time slot
    problem.addConstraint(AllDifferentConstraint(), team_matches) 

C3: In every time slot, there can be at most one match per location.

In [ ]:
from typing import Set, Tuple, Callable

# creates a 4-arg function that checks exactly the implication of constraint 6
# both in first half --> both reverses in second half with preserved order
# loop adds exactly one constraint instance per unordered match pairing (which reduces
# the number of redundant contraints). function is conservative for partial assignments:
# returns true as long as the condition is not relevant or had not yet been violated.
def make_c6_constraint(first_half: Set[int], second_half: Set[int]) -> Callable:
    def c6(s_m1, s_m2, s_m1r, s_m2r):
        # if both matches are scheduled in the first half, enforce ordering preservation
        if (s_m1 in first_half) and (s_m2 in first_half):
            # if they happen at the same slot, don't enforce ordering here (other constraints cover conflicts)
            if s_m1 == s_m2:
                return True
            # both reverses must be in second half
            if not (s_m1r in second_half and s_m2r in second_half):
                return False
            # preserve relative order: s_m1 < s_m2  iff s_m1r < s_m2r
            return (s_m1 < s_m2 and s_m1r < s_m2r) or (s_m2 < s_m1 and s_m2r < s_m1r)
        # otherwise the implication doesn't apply (still ok)
        return True
    return c6

# add constraints to the problem for all unordered pairs of distinct matches ---
# assumes `matches` is the list of directed match tuples and `problem` exists,
# and `first_half` / `second_half` are defined as sets of allowed slot numbers.

c6_fn = make_c6_constraint(first_half, second_half)

# iterate only over unordered pairs to avoid duplicate constraints
for i in range(len(matches)):
    for j in range(i + 1, len(matches)):
        m1 = matches[i]
        m2 = matches[j]
        m1r = (m1[1], m1[0])
        m2r = (m2[1], m2[0])
        # only add constraint if reverse variables exist in the problem (they should, since we used permutations)
        if m1r in matches and m2r in matches:
            problem.addConstraint(c6_fn, (m1, m2, m1r, m2r))

C6: If ⟨A, B⟩ is played before ⟨C, D⟩ in the first half of the season, then ⟨B, A⟩ should be played before ⟨D, C⟩ in the second half of the season.

In [ ]:
def check_constraint6(assignment, H):
    matches = list(assignment.key())
    for m1 in matches:
        for m2 in matches:
            if m1 == m2:
                continue
            m1r = (m1[1], m1[0])
            m2r = (m2[1], m2[0])
            # Need all four assigned to check the implication
            if m1 in assignment and m2 in assignment and m1r in assignment and m2r in assignment:
                r1 = assignment[m1]; r2 = assignment[m2]
                rr1 = assignment[m1r]; rr2 = assignment[m2r]
                # only relevant if both are in the first half and ordered
                if 1 <= r1 <= H and 1 <= r2 <= H and r1 < r2:
                    # then their reverses must be in the second half and ordered
                    if not (H + 1 <= rr1 <= 2 * H and H + 1 <= rr2 <= 2 * H and rr1 < rr2):
                        return False
    return True

## Solution

In [ ]:
def print_solution(sol):
    # TODO specify your own print function
    pass

In [161]:
solutions = problem.getSolutions()

In [167]:
print(f"Number of solutions: {len(solutions)}\n")
print_solution(solutions[0])

Number of solutions: 2

('A', 'B') in A at time 1
('A', 'D') in A at time 4
('B', 'C') in B at time 4
('C', 'D') in C at time 5
('A', 'C') in A at time 6
('B', 'D') in B at time 6
('B', 'A') in B at time 7
('C', 'B') in C at time 8
('D', 'A') in D at time 8
('D', 'C') in D at time 9
('C', 'A') in C at time 10
('D', 'B') in D at time 10

('A', 'B') in A at time 1
('A', 'D') in A at time 4
('B', 'C') in B at time 4
('C', 'D') in C at time 5
('A', 'C') in A at time 6
('B', 'D') in B at time 6
('B', 'A') in B at time 7
('C', 'B') in C at time 8
('D', 'A') in D at time 8
('D', 'C') in D at time 9
('C', 'A') in C at time 10
('D', 'B') in D at time 10
